In [1]:
import pandas as pd
from utils import get_path, group_cols
from datatable import dt, f, by

# Descrição

Neste notebook iremos formar a tabela deslocamento, com origem e destino.

# Tabelas base

## Procedimentos

In [2]:
path_proc = get_path('GESTANTES', 'procs_bienio.csv.gzip')
df_proc = dt.fread(path_proc)
df_proc

,bienio,parto_normal,origem,destino,distancia (km),tempo (min),dist_pond,tempo_pond,criticidade,capitais,…,origem_latitude,origem_longitude,destino_latitude,destino_longitude,count
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010464,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,260
1,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010472,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,282
2,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010480,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,3323
3,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010499,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,3368
4,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010502,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,964
5,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010510,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,832
6,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010529,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,1831
7,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010537,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,3707
8,0,0,CO/DF/53001/530010,CO/DF/53001/530010/0010545,0,0,0,0,0,1_1,…,−15.7795,−47.9297,−15.7795,−47.9297,728


## CNES

In [3]:
cols_cnes = {
  'CNES': 'id_destino',
  'Nome Fantasia': 'nome_destino',
}

In [4]:
path_cnes = get_path('LOCALIDADES', 'cnes.csv.gzip')
df_cnes = pd.read_csv(path_cnes, dtype={'CNES': str})
df_cnes = df_cnes.rename(columns=cols_cnes)[cols_cnes.values()]
df_cnes = dt.Frame(df_cnes)
df_cnes

,id_destino,nome_destino
,▪▪▪▪,▪▪▪▪
0,0002232,HOSPITAL E MATERNIDADE SANTA IZABEL
1,3151794,MATERNIDADE DE REFERENCIA ANA BRAGA
2,2323397,MATERNIDADE DONA EVANGELINA ROSA
3,2311682,HOSPITAL SANTA HELENA
4,0026794,HOSPITAL SOFIA FELDMAN
5,2020068,SES AP HOSPITAL DA MULHER
6,6627595,INSTITUTO DA MULHER DONA LINDU
7,2799278,HOSPITAL INACIA PINTO DOS SANTOS
8,0010480,HOSPITAL REGIONAL DE CEILANDIA


# Gerando tabela

In [5]:
cols_order = [
  'bienio',
  'parto_normal',
  'origem',
  'destino',
  'escala_origem',
  'escala_destino',
  'id_origem',
  'id_destino',
  'nome_origem',
  'nome_destino',
  'criticidade',
  'dist_pond',
  'tempo_pond',
  'count',
]

In [6]:
path_locs = get_path('LOCALIDADES', 'localidades.csv.gzip')

In [7]:
def get_flux_table(scale_origin, scale_destin, df=df_proc, df_cnes=df_cnes, path_locs=path_locs, cols_order=cols_order):
  cols_origem = {'codigo': 'origem', f'nome_{scale_origin}': 'nome_origem', ...: None}
  cols_destino = {'codigo': 'destino', f'nome_{scale_destin}': 'nome_destino', ...: None}
  df_locs_orig = dt.fread(path_locs, columns=cols_origem)
  df_locs_dest = dt.fread(path_locs, columns=cols_destino)
  df_locs_orig.key = 'origem'
  df_locs_dest.key = 'destino'
  cols_proc = ['bienio', 'parto_normal', 'origem', 'destino', 'criticidade', 'dist_pond', 'tempo_pond', 'count']
  df_flux = df[:, cols_proc]
  df_flux = df_flux[:, :, dt.join(df_locs_orig)]
  if scale_destin == 'cnes':
    df_locs_dest['nome_destino'] = ''
  else:
    df_flux['destino'] = dt.Frame(['/'.join(c.split('/')[:-1]) for c in df_flux['destino'].to_list()[0]])
  df_flux = df_flux[:, :, dt.join(df_locs_dest)]
  df_flux['escala_origem'] = scale_origin
  df_flux['escala_destino'] = scale_destin
  pos = {'cnes': 5, 'municipio': 4, 'regiao_saude': 3, 'uf': 2, 'regiao': 1}
  for ref in ['origem', 'destino']:
    i = pos[scale_origin] if ref == 'origem' else pos[scale_destin]
    codigos = df_flux[ref].to_list()[0]
    df_cods = ['/'.join(c.split('/')[:i]) for c in codigos]
    df_flux[ref] = dt.Frame(df_cods)
    df_flux[f'id_{ref}'] = dt.Frame([c.split('/')[-1] for c in df_flux[ref].to_list()[0]])
  df_flux = df_flux[:, cols_order]
  df_flux = df_flux[:, [dt.sum(f.count), dt.sum(f.dist_pond), dt.sum(f.tempo_pond)], by(df_flux.names[:-3])]
  df_total = df_flux[:, ['bienio', 'parto_normal', 'id_origem', 'id_destino', 'count']]
  df_total = df_total[:, dt.sum(f.count), by(df_total.names[:-1])]
  df_total.names = {'count': 'total'}
  df_total.key = df_total.names[:-1]
  df_flux = df_flux[:, :, dt.join(df_total)]
  df_flux[:, dt.update(percent = f.count / f.total * 100)]
  if scale_destin == 'cnes':
    df_cnes.key = 'id_destino'
    df_id_cnes = df_flux[f.escala_destino == 'cnes', ['id_destino']]
    df_id_cnes = df_id_cnes[:, :, dt.join(df_cnes)]
    df_flux[f.escala_destino == 'cnes', ['id_destino', 'nome_destino']] = df_id_cnes
  df_flux = df_flux.sort(f.origem, f.destino, f.criticidade, f.parto_normal, f.bienio)
  return df_flux

## REGSAU -> CNES

In [8]:
scale_origin = 'regiao_saude'
scale_destin = 'cnes'
df_regsau_cnes = get_flux_table(scale_origin, scale_destin)
df_regsau_cnes

,bienio,parto_normal,origem,destino,escala_origem,escala_destino,id_origem,id_destino,nome_origem,nome_destino,…,count,dist_pond,tempo_pond,total,percent
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,CO/DF/53001,CO/DF/53001/530010/0010464,regiao_saude,cnes,53001,0010464,Distrito Federal,HRAN,…,260,0,0,260,100
1,1,0,CO/DF/53001,CO/DF/53001/530010/0010464,regiao_saude,cnes,53001,0010464,Distrito Federal,HRAN,…,1954,0,0,1954,100
2,0,1,CO/DF/53001,CO/DF/53001/530010/0010464,regiao_saude,cnes,53001,0010464,Distrito Federal,HRAN,…,2171,0,0,2171,100
3,1,1,CO/DF/53001,CO/DF/53001/530010/0010464,regiao_saude,cnes,53001,0010464,Distrito Federal,HRAN,…,2521,0,0,2521,100
4,0,0,CO/DF/53001,CO/DF/53001/530010/0010472,regiao_saude,cnes,53001,0010472,Distrito Federal,HRG,…,282,0,0,282,100
5,1,0,CO/DF/53001,CO/DF/53001/530010/0010472,regiao_saude,cnes,53001,0010472,Distrito Federal,HRG,…,650,0,0,650,100
6,0,1,CO/DF/53001,CO/DF/53001/530010/0010472,regiao_saude,cnes,53001,0010472,Distrito Federal,HRG,…,2739,0,0,2739,100
7,1,1,CO/DF/53001,CO/DF/53001/530010/0010472,regiao_saude,cnes,53001,0010472,Distrito Federal,HRG,…,1981,0,0,1981,100
8,0,0,CO/DF/53001,CO/DF/53001/530010/0010480,regiao_saude,cnes,53001,0010480,Distrito Federal,HOSPITAL REGIONAL DE CEILANDIA,…,3323,0,0,3323,100


### Apenas RJ

In [9]:
def filter_rj(df):
  return ['SD/RJ' in c for c in df['origem'].to_list()[0]]

In [10]:
df_regsau_cnes[filter_rj(df_regsau_cnes), :]

,bienio,parto_normal,origem,destino,escala_origem,escala_destino,id_origem,id_destino,nome_origem,nome_destino,…,count,dist_pond,tempo_pond,total,percent
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1,SD/RJ/33001,ND/PB/25012/250690/6644996,regiao_saude,cnes,33001,6644996,Baia da Ilha Grande,HOSPITAL REGIONAL DE ITABAIANA,…,1,2485.74,1898.2,1,100
1,1,1,SD/RJ/33001,ND/PI/22005/220770/8015899,regiao_saude,cnes,33001,8015899,Baia da Ilha Grande,HOSPITAL ESTADUAL DIRCEU ARCOVERDE,…,1,3067.57,2331.66,1,100
2,0,0,SD/RJ/33001,SD/ES/32002/320245/2448203,regiao_saude,cnes,33001,2448203,Baia da Ilha Grande,HOSPITAL DE IBATIBA,…,2,1229.23,919.537,2,100
3,1,1,SD/RJ/33001,SD/ES/32002/320245/2448203,regiao_saude,cnes,33001,2448203,Baia da Ilha Grande,HOSPITAL DE IBATIBA,…,1,704.828,573.227,1,100
4,0,0,SD/RJ/33001,SD/RJ/33001/330010/2280868,regiao_saude,cnes,33001,2280868,Baia da Ilha Grande,HOSPITAL E MATERNIDADE DE ANGRA DOS REIS HMAR,…,533,0,0,538,99.0706
5,1,0,SD/RJ/33001,SD/RJ/33001/330010/2280868,regiao_saude,cnes,33001,2280868,Baia da Ilha Grande,HOSPITAL E MATERNIDADE DE ANGRA DOS REIS HMAR,…,1698,0,0,1743,97.4182
6,0,1,SD/RJ/33001,SD/RJ/33001/330010/2280868,regiao_saude,cnes,33001,2280868,Baia da Ilha Grande,HOSPITAL E MATERNIDADE DE ANGRA DOS REIS HMAR,…,1457,0,0,1490,97.7852
7,1,1,SD/RJ/33001,SD/RJ/33001/330010/2280868,regiao_saude,cnes,33001,2280868,Baia da Ilha Grande,HOSPITAL E MATERNIDADE DE ANGRA DOS REIS HMAR,…,1781,0,0,1822,97.7497
8,0,0,SD/RJ/33001,SD/RJ/33001/330010/2280868,regiao_saude,cnes,33001,2280868,Baia da Ilha Grande,HOSPITAL E MATERNIDADE DE ANGRA DOS REIS HMAR,…,5,323.197,312.422,538,0.929368


## REGSAU -> Município (RJ)

In [11]:
scale_origin = 'regiao_saude'
scale_destin = 'municipio'
df_regsau_mun = get_flux_table(scale_origin, scale_destin)
df_regsau_mun[filter_rj(df_regsau_mun), :]

,bienio,parto_normal,origem,destino,escala_origem,escala_destino,id_origem,id_destino,nome_origem,nome_destino,…,count,dist_pond,tempo_pond,total,percent
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1,SD/RJ/33001,ND/PB/25012/250690,regiao_saude,municipio,33001,250690,Baia da Ilha Grande,Itabaiana,…,1,2485.74,1898.2,1,100
1,1,1,SD/RJ/33001,ND/PI/22005/220770,regiao_saude,municipio,33001,220770,Baia da Ilha Grande,Parnaíba,…,1,3067.57,2331.66,1,100
2,0,0,SD/RJ/33001,SD/ES/32002/320245,regiao_saude,municipio,33001,320245,Baia da Ilha Grande,Ibatiba,…,2,1229.23,919.537,2,100
3,1,1,SD/RJ/33001,SD/ES/32002/320245,regiao_saude,municipio,33001,320245,Baia da Ilha Grande,Ibatiba,…,1,704.828,573.227,1,100
4,0,0,SD/RJ/33001,SD/RJ/33001/330010,regiao_saude,municipio,33001,330010,Baia da Ilha Grande,Angra dos Reis,…,649,0,0,665,97.594
5,1,0,SD/RJ/33001,SD/RJ/33001/330010,regiao_saude,municipio,33001,330010,Baia da Ilha Grande,Angra dos Reis,…,1775,0,0,1822,97.4204
6,0,1,SD/RJ/33001,SD/RJ/33001/330010,regiao_saude,municipio,33001,330010,Baia da Ilha Grande,Angra dos Reis,…,1783,0,0,1842,96.797
7,1,1,SD/RJ/33001,SD/RJ/33001/330010,regiao_saude,municipio,33001,330010,Baia da Ilha Grande,Angra dos Reis,…,1863,0,0,1905,97.7953
8,0,0,SD/RJ/33001,SD/RJ/33001/330010,regiao_saude,municipio,33001,330010,Baia da Ilha Grande,Angra dos Reis,…,16,1379.67,1676.66,665,2.40602


## REGSAU -> REGSAU (RJ)

In [12]:
scale_origin = 'regiao_saude'
scale_destin = 'regiao_saude'
df_regsau_regsau = get_flux_table(scale_origin, scale_destin)
df_regsau_regsau[filter_rj(df_regsau_regsau), :]

,bienio,parto_normal,origem,destino,escala_origem,escala_destino,id_origem,id_destino,nome_origem,nome_destino,…,count,dist_pond,tempo_pond,total,percent
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1,SD/RJ/33001,ND/PB/25012,regiao_saude,regiao_saude,33001,25012,Baia da Ilha Grande,12ª Região,…,1,2485.74,1898.2,1,100
1,1,1,SD/RJ/33001,ND/PI/22005,regiao_saude,regiao_saude,33001,22005,Baia da Ilha Grande,Planície Litorânea,…,1,3067.57,2331.66,1,100
2,0,0,SD/RJ/33001,SD/ES/32002,regiao_saude,regiao_saude,33001,32002,Baia da Ilha Grande,Metropolitana,…,2,1229.23,919.537,2,100
3,1,1,SD/RJ/33001,SD/ES/32002,regiao_saude,regiao_saude,33001,32002,Baia da Ilha Grande,Metropolitana,…,1,704.828,573.227,1,100
4,0,0,SD/RJ/33001,SD/RJ/33001,regiao_saude,regiao_saude,33001,33001,Baia da Ilha Grande,Baia da Ilha Grande,…,923,0,0,939,98.2961
5,1,0,SD/RJ/33001,SD/RJ/33001,regiao_saude,regiao_saude,33001,33001,Baia da Ilha Grande,Baia da Ilha Grande,…,2445,0,0,2500,97.8
6,0,1,SD/RJ/33001,SD/RJ/33001,regiao_saude,regiao_saude,33001,33001,Baia da Ilha Grande,Baia da Ilha Grande,…,2714,0,0,2773,97.8723
7,1,1,SD/RJ/33001,SD/RJ/33001,regiao_saude,regiao_saude,33001,33001,Baia da Ilha Grande,Baia da Ilha Grande,…,2564,0,0,2618,97.9374
8,0,0,SD/RJ/33001,SD/RJ/33001,regiao_saude,regiao_saude,33001,33001,Baia da Ilha Grande,Baia da Ilha Grande,…,16,1379.67,1676.66,939,1.70394


## Município -> Município (RJ)

In [13]:
scale_origin = 'municipio'
scale_destin = 'municipio'
df_mun_mun = get_flux_table(scale_origin, scale_destin)
df_mun_mun_rj = df_mun_mun[filter_rj(df_mun_mun), :]
df_mun_mun_rj

,bienio,parto_normal,origem,destino,escala_origem,escala_destino,id_origem,id_destino,nome_origem,nome_destino,…,count,dist_pond,tempo_pond,total,percent
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1,SD/RJ/33001/330010,ND/PI/22005/220770,municipio,municipio,330010,220770,Angra dos Reis,Parnaíba,…,1,3067.57,2331.66,1,100
1,0,0,SD/RJ/33001/330010,SD/ES/32002/320245,municipio,municipio,330010,320245,Angra dos Reis,Ibatiba,…,2,1229.23,919.537,2,100
2,0,0,SD/RJ/33001/330010,SD/RJ/33001/330010,municipio,municipio,330010,330010,Angra dos Reis,Angra dos Reis,…,649,0,0,649,100
3,1,0,SD/RJ/33001/330010,SD/RJ/33001/330010,municipio,municipio,330010,330010,Angra dos Reis,Angra dos Reis,…,1775,0,0,1775,100
4,0,1,SD/RJ/33001/330010,SD/RJ/33001/330010,municipio,municipio,330010,330010,Angra dos Reis,Angra dos Reis,…,1783,0,0,1783,100
5,1,1,SD/RJ/33001/330010,SD/RJ/33001/330010,municipio,municipio,330010,330010,Angra dos Reis,Angra dos Reis,…,1863,0,0,1863,100
6,1,1,SD/RJ/33001/330010,SD/RJ/33001/330260,municipio,municipio,330010,330260,Angra dos Reis,Mangaratiba,…,1,56.7885,47.1,1,100
7,1,0,SD/RJ/33001/330010,SD/RJ/33001/330380,municipio,municipio,330010,330380,Angra dos Reis,Paraty,…,8,768.343,992.173,8,100
8,1,1,SD/RJ/33001/330010,SD/RJ/33001/330380,municipio,municipio,330010,330380,Angra dos Reis,Paraty,…,11,1056.47,1364.24,11,100


## UF -> Região

In [14]:
scale_origin = 'uf'
scale_destin = 'regiao'
df_uf_regsau = get_flux_table(scale_origin, scale_destin)
# df_uf_regsau[filter_rj(df_uf_regsau), :]
df_uf_regsau

,bienio,parto_normal,origem,destino,escala_origem,escala_destino,id_origem,id_destino,nome_origem,nome_destino,…,count,dist_pond,tempo_pond,total,percent
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,0,CO/DF,CO,uf,regiao,DF,CO,Distrito Federal,Centro-Oeste,…,20906,0,0,20910,99.9809
1,1,0,CO/DF,CO,uf,regiao,DF,CO,Distrito Federal,Centro-Oeste,…,19949,0,0,19990,99.7949
2,0,1,CO/DF,CO,uf,regiao,DF,CO,Distrito Federal,Centro-Oeste,…,40969,0,0,40981,99.9707
3,1,1,CO/DF,CO,uf,regiao,DF,CO,Distrito Federal,Centro-Oeste,…,33309,0,0,33351,99.8741
4,0,0,CO/DF,CO,uf,regiao,DF,CO,Distrito Federal,Centro-Oeste,…,4,305.25,265.447,20910,0.0191296
5,1,0,CO/DF,CO,uf,regiao,DF,CO,Distrito Federal,Centro-Oeste,…,41,8201.44,6862.07,19990,0.205103
6,0,1,CO/DF,CO,uf,regiao,DF,CO,Distrito Federal,Centro-Oeste,…,12,3103.87,2606.93,40981,0.0292819
7,1,1,CO/DF,CO,uf,regiao,DF,CO,Distrito Federal,Centro-Oeste,…,42,6997.03,6154.23,33351,0.125933
8,0,0,CO/DF,ND,uf,regiao,DF,ND,Distrito Federal,Nordeste,…,2,2535.68,1909.4,2,100


# Exportando tabela

In [15]:
output_flux = get_path('GESTANTES', 'flux_mun_mun_rj.csv.gzip')
df_mun_mun_rj.to_csv(output_flux)

In [16]:
dt.fread(output_flux)

,bienio,parto_normal,origem,destino,escala_origem,escala_destino,id_origem,id_destino,nome_origem,nome_destino,…,count,dist_pond,tempo_pond,total,percent
,▪,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,1,1,SD/RJ/33001/330010,ND/PI/22005/220770,municipio,municipio,330010,220770,Angra dos Reis,Parnaíba,…,1,3067.57,2331.66,1,100
1,0,0,SD/RJ/33001/330010,SD/ES/32002/320245,municipio,municipio,330010,320245,Angra dos Reis,Ibatiba,…,2,1229.23,919.537,2,100
2,0,0,SD/RJ/33001/330010,SD/RJ/33001/330010,municipio,municipio,330010,330010,Angra dos Reis,Angra dos Reis,…,649,0,0,649,100
3,1,0,SD/RJ/33001/330010,SD/RJ/33001/330010,municipio,municipio,330010,330010,Angra dos Reis,Angra dos Reis,…,1775,0,0,1775,100
4,0,1,SD/RJ/33001/330010,SD/RJ/33001/330010,municipio,municipio,330010,330010,Angra dos Reis,Angra dos Reis,…,1783,0,0,1783,100
5,1,1,SD/RJ/33001/330010,SD/RJ/33001/330010,municipio,municipio,330010,330010,Angra dos Reis,Angra dos Reis,…,1863,0,0,1863,100
6,1,1,SD/RJ/33001/330010,SD/RJ/33001/330260,municipio,municipio,330010,330260,Angra dos Reis,Mangaratiba,…,1,56.7885,47.1,1,100
7,1,0,SD/RJ/33001/330010,SD/RJ/33001/330380,municipio,municipio,330010,330380,Angra dos Reis,Paraty,…,8,768.343,992.173,8,100
8,1,1,SD/RJ/33001/330010,SD/RJ/33001/330380,municipio,municipio,330010,330380,Angra dos Reis,Paraty,…,11,1056.47,1364.24,11,100


In [19]:
df_rj = pd.read_csv(output_flux)
output_rj = output_flux.replace('.csv.gzip', '_rj.xlsx')
df_rj.to_excel(output_rj, index=False)